In [1]:
import bs4
import json
from enum import Enum, EnumMeta
import numpy as np
import pprint

In [2]:
class Payload:  
    def __init__(self, id, weight):
        self._id = id
        self._weight = weight

In [3]:
class Effector:
    def __init__(self, id, type, position, payload):
        self._id = id
        self._type = type
        self._position = position
        self._payload = payload

### DEFINITION D'UN MOUVEMENT ET SES OBJETS ASSOCIES

In [4]:
class Point:
    def __init__(self, cnt, speed, path, position):
        self._cnt = cnt
        self._speed = speed
        self._position = position
        self._path = path

class Position:
    def __init__(self, pvector, ptype, e1, config):
        self._vector = pvector
        self._type = ptype
        self._config=config
        self._e1 = e1

class Path(Enum):
    CIRCULAR='C'
    LINEAR='L'
    JOINT='J'
    
class PositionType(Enum):
    JOINT='jnt'
    CARTESIAN='crt'
    
class PositionCrt(Position):
    def __init__(self, pvector, e1, config):
        super(PositionCrt, self).__init__(pvector, PositionType.CARTESIAN, e1, config)
    
    def to_dict(self):
        keys = ['x', 'y','z', 'w', 'p', 'r']
        tl = [(keys[i], float(val)) for i, val in enumerate(self._vector.tolist())]
        crt_dict = self.__dict__.copy()
        crt_dict['_vector'] = dict(tl)
        return crt_dict
    
    def get(self):
        keys = ['x', 'y','z', 'w', 'p', 'r']
        tl = [(keys[i], float(val)) for i, val in enumerate(self._vector.tolist())]
        return {'vector':dict(tl), 'type':self._type.value, 'config':Serializer.to_dict(self._config), 'e1':float(self._e1)}
            
class PositionJoint(Position):
    def __init__(self, pvector, e1):
        super(PositionJoint, self).__init__(pvector, PositionType.JOINT, e1, None)
    
    def to_dict(self):
        keys = ['j1', 'j2','j3', 'j4', 'j5', 'j6']
        tl = [(keys[i], float(val)) for i, val in enumerate(self._vector.tolist())]
        jnt_dict = self.__dict__.copy()
        jnt_dict['_vector'] = dict(tl)
        return jnt_dict
        
class ForeArm(Enum):
    UP='U'
    DOWN='D'
    
class Wrist(Enum):
    FLIP='F'
    NOFLIP='N'

class Arm(Enum):
    TOWARD='T'
    BACKWARD='D'

class Configuration:
    def __init__(self, wrist, forearm, arm): #left
        self._forarm = forearm
        self._wrist = wrist
        #self._left = left
        self._arm = arm
        self._j4 = 0
        self._j5 = 0
        self._j6 = 0

class Movement:
    def __init__(self, uf, ut, points):
        self._uf = uf
        self._ut = ut
        self._points = points

### DEFINITION DES TYPE D'ACTIONS

In [5]:
class EnumLevelInterface(EnumMeta):
    
    def __getitem__(self, name):
        dot = name.find('.')
        if dot!=-1:
            cindex = name[:dot]
            oindex = name[(dot+1):]
            e = super().__getitem__(cindex).value
            return e.__getitem__(oindex)
        else:
            e = super().__getitem__(name)
            if EnumPriorityInterface in e.__class__.__bases__:
                return e
            else:
                return e.value
    
class EnumPriorityInterface(Enum, metaclass=EnumLevelInterface):    
    def __init__(self, priority, name):
        self._priority = priority
        self._name = name  
    @property
    def priority(self):
        return self._priority
    @property
    def name(self):
        return self._name

class ActionArmMove(EnumPriorityInterface, metaclass=EnumLevelInterface):
    APPROACH = (50, 'MOVE.ARM.APPROACH')
    CLEARANCE = (50, 'MOVE.ARM.CLEARANCE')
    WORK = (60, 'MOVE.ARM.WORK')

class ActionStationMove(EnumPriorityInterface, metaclass=EnumLevelInterface):
    WORK = (40, 'MOVE.STATION.WORK')
    HOME = (10, 'MOVE.STATION.HOME')
    
class ActionLoad(EnumPriorityInterface, metaclass=EnumLevelInterface):
    EFFECTOR=(20, 'LOAD.EFFECTOR')
    TOOL=(30, 'LOAD.TOOL')

class ActionUnload(EnumPriorityInterface, metaclass=EnumLevelInterface):
    EFFECTOR=(20, 'UNLOAD.EFFECTOR')
    TOOL=(30, 'UNLOAD.TOOL')

class ActionWork(EnumPriorityInterface, metaclass=EnumLevelInterface):
    DRILL=(70, 'WORK.DRILL')
    FASTEN=(80, 'WORK.FASTEN')
    
class ActionMove(Enum, metaclass=EnumLevelInterface):
    ARM = ActionArmMove
    STATION = ActionStationMove
    
class ActionType(Enum, metaclass=EnumLevelInterface):
    LOAD = ActionLoad
    UNLOAD = ActionUnload
    MOVE = ActionMove
    WORK = ActionWork

### DEFINITION DE L'OBJET ACTION

In [52]:
class Action:
    def __init__(self, id, atype, definition, description, dependencies, anext):
        self._id = id
        self._type = atype
        self._definition = definition
        self._dependencies = dependencies
        self._next = anext
        self._description = description
        
    def __repr__(self):
        return Serializer.serialize(self)

### CLASS DE SERIALISATION

In [50]:
class Serializer:
    @staticmethod
    def serialize(obj, form="json", database=False):
        if(form =="json"):
            so = Serializer.to_dict(obj, database)
            return json.dumps(so)
    
    @staticmethod
    def to_dict(obj, database=False):
        if obj.__class__.__module__ == 'builtins':
            if type(obj) == list:
                l=[] 
                for va in obj:
                    l.append(Serializer.to_dict(va, database))
                return l
            elif type(obj) == dict:
                d={} 
                for key, val in obj.items():
                    k = key.replace('_','')
                    v = Serializer.to_dict(val, database)   
                    d[k]=v 
                return d
            else :
                return Serializer.__cast_val(obj)
            
        elif getattr(obj, 'to_dict', None):
            return  Serializer.to_dict(obj.to_dict(), database)
        
        elif EnumPriorityInterface in obj.__class__.__bases__ or Enum in obj.__class__.__bases__:
            if database : 
                return obj.name
            else:
                return obj.value
        else:
            so = {}
            for key, val in obj.__dict__.items():
                k = key.replace('_','')
                v = Serializer.to_dict(val, database)   
                so[k]=v
            return so
        
        
    @staticmethod
    def __cast_val(val):
        if isinstance(val, str) and val.isnumeric():
            return int(val) if val.isdecimal() else float(val)
        else:
            return val

### TEST DE LA SERIALISATION

In [8]:
# definition d'un vecteur representant la position haute de bras
no_work_arm_joint_vector = np.array([0.0,25.0,40.0,113.0,-307.0])

In [9]:
# definition de la position home (bras postion no_work + e1 = 0) 
home_position = PositionJoint(no_work_arm_joint_vector, e1=0.0)

# definition du mouvement home 
home_point = Point(100, 100, Path.JOINT, home_position)
home_movement = Movement(1,1,[home_point])

# definition de l'action mouvement vers home
home_action = Action(0, ActionType['MOVE']['STATION']['HOME'], home_movement, 'move to home position', [], [])

In [10]:
# definition de la position station_1 (bras postion no_work + e1 = -300) 
station_1_e1 = -300.0
station_1_position = PositionJoint(no_work_arm_joint_vector, e1=station_1_e1)

# definition du mouvement moveto station1
station_1_point = Point(100, 100, Path.JOINT, station_1_position)
station_1_movement = Movement(1,1,[station_1_point])

# definition de l'action mouvement vers home
station_1_action = Action(0, ActionType['MOVE']['STATION']['WORK'], station_1_movement, 'move to station 1 position', [], [])

In [11]:
work1_config = Configuration(Wrist.FLIP, ForeArm.UP, Arm.TOWARD)
work_1_point = Point(100,100,Path['LINEAR'], PositionCrt(np.array([0.0,25.0,40.0,113.0,-307]), e1=-300.0, config=work1_config))
work_1_mvt = Movement(1,1, [work_1_point])
work_1_action = Action(1, ActionType['MOVE']['ARM']['WORK'], work_1_mvt, "move to point 1", [], [])

In [12]:
Serializer.serialize(work_1_action, database=False)

'{"id": 1, "type": [60, "MOVE.ARM.WORK"], "definition": {"uf": 1, "ut": 1, "points": [{"cnt": 100, "speed": 100, "position": {"vector": {"x": 0.0, "y": 25.0, "z": 40.0, "w": 113.0, "p": -307.0}, "type": "crt", "config": {"forarm": "U", "wrist": "F", "arm": "T", "j4": 0, "j5": 0, "j6": 0}, "e1": -300.0}, "path": "L"}]}, "dependencies": [], "next": [], "description": "move to point 1"}'

In [13]:
Serializer.serialize(home_action, database=True)

'{"id": 0, "type": "MOVE.STATION.HOME", "definition": {"uf": 1, "ut": 1, "points": [{"cnt": 100, "speed": 100, "position": {"vector": {"j1": 0.0, "j2": 25.0, "j3": 40.0, "j4": 113.0, "j5": -307.0}, "type": "JOINT", "config": null, "e1": 0.0}, "path": "JOINT"}]}, "dependencies": [], "next": [], "description": "move to home position"}'

In [14]:
Serializer.serialize(station_1_position, database=True)

'{"vector": {"j1": 0.0, "j2": 25.0, "j3": 40.0, "j4": 113.0, "j5": -307.0}, "type": "JOINT", "config": null, "e1": -300.0}'

### PARSING XML ET CREATION DU PROCESS

In [15]:
import lxml

In [16]:
with open('BuildProcess.xml', 'r') as f:
    txt = f.readlines()
    txt = ''.join(txt)

In [21]:
# racine du fichier
xbp = bs4.BeautifulSoup(txt, 'lxml')

In [22]:
phases = xbp.find_all('phase')

In [23]:
phases[1]

<phase fastname="asna2392-3-04.178" id="2" info="Drill" mode="crt" type="position"><point approx="fine" cnt="100" e1="700" flip="true" front="true" id="1" left="false" p="-90" r="0" speed="100" t4="0" t5="0" t6="0" type="linear" uf="1" up="true" ut="1" w="90" x="810.3" y="1360" z="-27.1"></point></phase>

In [24]:
points = phases[1].find_all('point')

In [25]:
points[0]

<point approx="fine" cnt="100" e1="700" flip="true" front="true" id="1" left="false" p="-90" r="0" speed="100" t4="0" t5="0" t6="0" type="linear" uf="1" up="true" ut="1" w="90" x="810.3" y="1360" z="-27.1"></point>

In [26]:
def getPath(xml_path):
    path = None
    if xml_path =="linear":
        path = Path.LINEAR
    elif xml_path == "joint":
        path = Path.JOINT
    else:
        path = Path.CIRCULAR
    return path


#vecteur de position
def getMovementFromXMLPhases(bsxml_phase):
    
    #get phase attributes
    phat = bsxml_phase.attrs
    
    #get bs points 
    points = bsxml_phase.find_all('point')
    
    ps=[]
    uf = points[0].attrs.get('uf')
    ut = points[0].attrs.get('ut')
    
    for p in points:
        ps.append(getPointFromXMLPoint(p, phat['mode']))

    return Movement(uf, ut, ps)


def getPointFromXMLPoint(point, pos_type):
    
    poat = point.attrs
    path = getPath(poat['type'])
    
    cnt = poat.get('cnt')
    speed = poat.get('speed')
    
    vector = np.array([
            poat['x'],
            poat['y'],
            poat['z'],
            poat['w'],
            poat['p'],
            poat['r']])

    config = None
    
    if poat.get("front") and poat.get("up") and poat.get("left") and poat.get("flip"):
        config = Configuration(Wrist.FLIP if bool(poat.get("flip")) else Wrist.NOFLIP,
                               ForeArm.UP if bool(poat.get("up")) else ForeArm.DOWN,
                               Arm.TOWARD if bool(poat.get("front")) else Arm.BACKWARD
                               )

    e1 = poat.get('e1')

    if pos_type =="joint":
        position = PositionJoint(vector, e1)
    elif pos_type == 'crt':
        position = PositionCrt(vector, e1, config)
    else :
        position = None
    
    return Point(cnt, speed, path, position)
    

def getActionFromXMLStation(bsxml_station):
    
    actions = []
    phases = bsxml_station.find_all('phase')
    
    for p in phases:
        mov = getMovementFromXMLPhases(p)
        actions.append(Action(0, ActionType['MOVE'][''], mov, p.attrs['info'], [], [])) 
        #id, atype, definition, description, dependencies, anext
        
    return actions
    

In [27]:
steps = xbp.find_all('step')
stations = steps[0].find_all('station')

la = getActionFromXMLStation(stations[0])

s = []
for a in la:
    s.append(Serializer.serialize(a))

In [ ]:
s ="["+ ",".join(s)+"]"

## CREATION DES ACTIONS POUR ENREGISTREMENT

In [25]:
import pandas

### RAIL 1

In [26]:
df = pandas.read_csv('rail1_data.csv')

In [27]:
df.head()

,mvt,id,order,points,type,x_j1,y_j2,z_j3,w_j4,p_j5,r_j6,e1,wrist,forearm,arm,conf_j1,conf_j4,conf_j6
0,home,NaN,NaN,home,JOINT,0.0,25.00,-30.0,0.00,113.00,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,station,NaN,NaN,station 1,JOINT,0.0,25.00,-30.0,0.00,113.00,0.00,-300.0,NaN,NaN,NaN,NaN,NaN,NaN
2,approach,NaN,NaN,12,JOINT,0.0,25.00,-30.0,0.00,113.00,0.00,-300.0,NaN,NaN,NaN,NaN,NaN,NaN
3,approach,NaN,NaN,13,JOINT,24.6,37.01,-12.6,-71.07,115.06,27.86,-300.0,NaN,NaN,NaN,NaN,NaN,NaN
4,approach,NaN,NaN,14,JOINT,24.6,37.01,-12.6,-71.07,115.06,-29.74,-300.0,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
def get_position(data):
    vector = np.array([data.x_j1, data.y_j2, data.z_j3, data.w_j4, data.p_j5, data.r_j6])
    if(data.type == 'JOINT'):
        return PositionJoint(vector, data.e1)
    else:
        wrist = Wrist.FLIP if data.wrist == 'F' else Wrist.NOFLIP
        forearm = ForeArm.UP if data.forearm == 'U' else ForeArm.DOWN
        arm = Arm.TOWARD if data.arm == 'T' else Arm.BACKWARD
        
        config = Configuration(wrist, forearm, arm)
        return PositionCrt(vector, data.e1, config)

    
def getMovementFromDf(df_data):
    points = []
    for i, d in df_data.iterrows():
        position = get_position(d)
        if d.type == 'JOINT':
            path = Path.JOINT
        elif d.type == 'CARTESIAN':
            path = Path.LINEAR
        else:
            path =Path.CIRCULAR
        
        points.append(Point(100, 100, path, position))
    return Movement(ut=1, uf=1, points=points)


#Action(1, ActionType['MOVE']['ARM']['WORK'], work_1_mvt, "move to point 1", [], [])
#Point(100,100,Path['LINEAR'], PositionCrt(np.array([0.0,25.0,40.0,113.0,-307]), e1=-300.0, config=work1_config))

#### APPROCHE

In [33]:
app_mvt_df = df[df.mvt == 'approach']
app_mvt_df = app_mvt_df.dropna(axis=1)
app_mvt_df.index = app_mvt_df.points
app_mvt_df = app_mvt_df.drop(['points','mvt'], axis=1)

In [34]:
app_mvt.head()

,type,x_j1,y_j2,z_j3,w_j4,p_j5,r_j6,e1
points,,,,,,,,
12,JOINT,0.00,25.00,-30.00,0.00,113.00,0.00,-300.0
13,JOINT,24.60,37.01,-12.60,-71.07,115.06,27.86,-300.0
14,JOINT,24.60,37.01,-12.60,-71.07,115.06,-29.74,-300.0
15,JOINT,30.79,40.03,-5.71,-88.94,115.58,-13.89,-300.0
16,JOINT,41.35,45.74,7.34,-80.60,125.00,51.18,-300.0


In [57]:
app_mov = getMovementFromDf(app_mvt_df)
app_action = Action(1, ActionType['MOVE.ARM.APPROACH'], app_mov, 'approach movement for rail 1 web fasteners', [], [])
app_action

{"id": 1, "type": [50, "MOVE.ARM.APPROACH"], "definition": {"uf": 1, "ut": 1, "points": [{"cnt": 100, "speed": 100, "position": {"vector": {"j1": 0.0, "j2": 25.0, "j3": -30.0, "j4": 0.0, "j5": 113.0, "j6": 0.0}, "type": "jnt", "config": null, "e1": -300.0}, "path": "J"}, {"cnt": 100, "speed": 100, "position": {"vector": {"j1": 24.6, "j2": 37.01, "j3": -12.6, "j4": -71.07, "j5": 115.06, "j6": 27.86}, "type": "jnt", "config": null, "e1": -300.0}, "path": "J"}, {"cnt": 100, "speed": 100, "position": {"vector": {"j1": 24.6, "j2": 37.01, "j3": -12.6, "j4": -71.07, "j5": 115.06, "j6": -29.74}, "type": "jnt", "config": null, "e1": -300.0}, "path": "J"}, {"cnt": 100, "speed": 100, "position": {"vector": {"j1": 30.79, "j2": 40.03, "j3": -5.71, "j4": -88.94, "j5": 115.58, "j6": -13.89}, "type": "jnt", "config": null, "e1": -300.0}, "path": "J"}, {"cnt": 100, "speed": 100, "position": {"vector": {"j1": 41.35, "j2": 45.74, "j3": 7.34, "j4": -80.6, "j5": 125.0, "j6": 51.18}, "type": "jnt", "config"

#### CLEARANCE

In [38]:
clear_mvt_df = df[df.mvt == 'clearance']
clear_mvt_df = clear_mvt_df.dropna(axis=1)
clear_mvt_df.index = clear_mvt_df.points
clear_mvt_df = clear_mvt_df.drop(['points','mvt'], axis=1)
clear_mvt_df.head()

,type,x_j1,y_j2,z_j3,w_j4,p_j5,r_j6,e1
points,,,,,,,,
10,JOINT,43.22,49.37,1.06,-129.16,106.21,41.02,-300.0
11,JOINT,44.99,46.97,10.11,-129.98,116.76,50.96,-300.0
17,JOINT,45.13,42.93,18.94,-121.71,112.68,53.40,-300.0
16,JOINT,41.35,45.74,7.34,-80.60,125.00,51.18,-300.0
15,JOINT,30.79,40.03,-5.71,-88.94,115.58,-13.89,-300.0


In [53]:
clear_mvt = getMovementFromDf(clear_mvt_df)
clear_action = Action(2, ActionType['MOVE.ARM.CLEARANCE'], clear_mvt, 'clearance movement for rail 1 web fasteners', [], [])

In [56]:
clear_action

{"id": 2, "type": [50, "MOVE.ARM.CLEARANCE"], "definition": {"uf": 1, "ut": 1, "points": [{"cnt": 100, "speed": 100, "position": {"vector": {"j1": 43.22, "j2": 49.37, "j3": 1.06, "j4": -129.16, "j5": 106.21, "j6": 41.02}, "type": "jnt", "config": null, "e1": -300.0}, "path": "J"}, {"cnt": 100, "speed": 100, "position": {"vector": {"j1": 44.99, "j2": 46.97, "j3": 10.11, "j4": -129.98, "j5": 116.76, "j6": 50.96}, "type": "jnt", "config": null, "e1": -300.0}, "path": "J"}, {"cnt": 100, "speed": 100, "position": {"vector": {"j1": 45.13, "j2": 42.93, "j3": 18.94, "j4": -121.71, "j5": 112.68, "j6": 53.4}, "type": "jnt", "config": null, "e1": -300.0}, "path": "J"}, {"cnt": 100, "speed": 100, "position": {"vector": {"j1": 41.35, "j2": 45.74, "j3": 7.34, "j4": -80.6, "j5": 125.0, "j6": 51.18}, "type": "jnt", "config": null, "e1": -300.0}, "path": "J"}, {"cnt": 100, "speed": 100, "position": {"vector": {"j1": 30.79, "j2": 40.03, "j3": -5.71, "j4": -88.94, "j5": 115.58, "j6": -13.89}, "type": "jn